In [ ]:
!pip install hyperas

In [2]:
import h5py
from os.path import join,exists
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.optimizers import RMSprop
from hyperas.distributions import choice
from hyperas import optim
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from random import randint
from keras import backend as K
import os
import numpy as np
from hyperopt import Trials, STATUS_OK, tpe
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve,roc_auc_score
K.set_image_data_format('channels_first')

In [3]:
os.chdir('/content/drive/My Drive/Colab Notebooks/synopsys-2022/models/classification')

In [4]:
def create_model(X_train, Y_train, X_test, Y_test):
    W_maxnorm = 3
    DROPOUT = {{choice([0.3,0.5,0.7])}}
    # DROPOUT=0.3

    model = Sequential()
    model.add(Conv2D(32, (1, 5), padding='same', input_shape=(20, 1, 20),activation='relu',kernel_constraint=maxnorm(W_maxnorm)))
    model.add(MaxPool2D(pool_size=(1, 2),strides=(1,2)))
    model.add(Conv2D(64, (1, 5), padding='same', activation='relu',kernel_constraint=maxnorm(W_maxnorm)))
    model.add(MaxPool2D(pool_size=(1, 2),strides=(1,2)))
    model.add(Flatten())

    model.add(Dense(16,activation='relu',kernel_constraint=maxnorm(W_maxnorm)))
    model.add(Dropout(DROPOUT))
    model.add(Dense(2,kernel_constraint=maxnorm(W_maxnorm),activation='softmax'))
    # model.add(Activation('softmax'))

    myoptimizer = RMSprop(learning_rate={{choice([1e-1,0.01,0.001,0.0001,1e-5])}}, rho=0.9, epsilon=1e-06)
    mylossfunc = 'binary_crossentropy'
    model.compile(loss=mylossfunc, optimizer=myoptimizer, metrics=['accuracy'])
    # model.compile(loss=RMSprop, optimizer=myoptimizer,metrics=['accuracy'])
    result = model.fit(X_train, Y_train, batch_size=100, epochs=5,validation_split=0.1,verbose=False)

    val_loss = np.amax(result.history['val_loss'])

    # score, acc = model.evaluate(X_test,Y_test, verbose=False)
    # model_arch = 'MODEL_ARCH'
    # bestaccfile = join('TOPDIR',model_arch,model_arch+'_hyperbestacc')
    # reportAcc(acc,score,bestaccfile)

    return {'loss': val_loss, 'status': STATUS_OK,'model':model}

In [5]:
def train_data():
  data_train = h5py.File('./data/train.h5.batch1','r')
  X_train = np.array(data_train['data'])
  Y_train = np.array(data_train['label'])
  X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train)

  data_test = h5py.File('./data/test.h5.batch1', 'r')
  X_test = np.array(data_test['data'])
  Y_test = np.array(data_test['label'])
  X_test_shuffled, Y_test_shuffled = shuffle(X_test, Y_test)
  return X_train_shuffled, Y_train_shuffled, X_test_shuffled, np.fliplr(Y_test_shuffled)

In [6]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=train_data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='seq_32x2_16',
                                          verbose=False)

100%|██████████| 10/10 [03:13<00:00, 19.31s/it, best loss: 0.15242089331150055]


In [7]:
X_train, Y_train, X_test, Y_test = train_data()
best_model.fit(X_train, Y_train, batch_size=100, epochs=20,validation_split=(0.1))

Epoch 1/20
571/571 [==============================] - 3s 6ms/step - loss: 0.2622 - accuracy: 0.9245 - val_loss: 0.2321 - val_accuracy: 0.9336
Epoch 2/20
571/571 [==============================] - 3s 6ms/step - loss: 0.2513 - accuracy: 0.9302 - val_loss: 0.2237 - val_accuracy: 0.9369
Epoch 3/20
571/571 [==============================] - 3s 6ms/step - loss: 0.2426 - accuracy: 0.9341 - val_loss: 0.2196 - val_accuracy: 0.9404
Epoch 4/20
571/571 [==============================] - 3s 6ms/step - loss: 0.2358 - accuracy: 0.9369 - val_loss: 0.2130 - val_accuracy: 0.9427
Epoch 5/20
571/571 [==============================] - 3s 6ms/step - loss: 0.2287 - accuracy: 0.9381 - val_loss: 0.2087 - val_accuracy: 0.9442
Epoch 6/20
571/571 [==============================] - 3s 6ms/step - loss: 0.2237 - accuracy: 0.9400 - val_loss: 0.2053 - val_accuracy: 0.9443
Epoch 7/20
571/571 [==============================] - 3s 6ms/step - loss: 0.2206 - accuracy: 0.9413 - val_loss: 0.2055 - val_accuracy: 0.9457
Epoch 

In [8]:
y_pred = best_model.predict(X_test)
roc_auc_score(Y_test,y_pred)

0.9676394363728871

In [10]:
best_model.save('./seq_32x2_16')

INFO:tensorflow:Assets written to: ./seq_32x2_16/assets
